In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tflearn

In [ ]:
import sklearn
import seaborn as sns
import time
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
import datetime
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.mixture import GaussianMixture
from scipy import stats
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.linear_model import RidgeCV, ElasticNetCV, LassoCV
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA
from sklearn.preprocessing import LabelEncoder, StandardScaler #transforms categorical into numbers
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, make_scorer, f1_score, confusion_matrix
import xgboost
from sklearn.cluster import KMeans
import zipfile
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm


%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, FixedFormatter
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
!pip install tflearn

In [ ]:
LR = 1e-3
MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '2conv-basic')
RANDOM_SEED = 42
IMG_WIDTH = 150
IMG_HEIGHT = 150
IMG_CHANNELS = 3

IMG_SHAPE = [IMG_WIDTH, IMG_HEIGHT]
IMG_LENGTH = IMG_WIDTH * IMG_HEIGHT

def get_model_baseline():
  convnet = input_data(shape=[None, IMG_WIDTH, IMG_HEIGHT, 3], name='input')

  convnet = conv_2d(convnet, 32, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = conv_2d(convnet, 64, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = fully_connected(convnet, 1024, activation='relu')
  convnet = dropout(convnet, 0.8)

  convnet = fully_connected(convnet, 2, activation='softmax')
  convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

  return tflearn.DNN(convnet, tensorboard_dir='log')

def reset_graph(seed= RANDOM_SEED):
    tf.compat.v1.reset_default_graph()
    tf.random.set_seed(seed)
    np.random.seed(seed)

In [ ]:
def get_model_v2():
  convnet = input_data(shape=[None, IMG_WIDTH, IMG_HEIGHT, 3], name='input')

  convnet = conv_2d(convnet, 32, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = conv_2d(convnet, 64, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = conv_2d(convnet, 128, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = conv_2d(convnet, 64, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = conv_2d(convnet, 32, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = fully_connected(convnet, 1024, activation='relu')
  convnet = dropout(convnet, 0.8)

  convnet = fully_connected(convnet, 2, activation='softmax')
  convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

  return tflearn.DNN(convnet, tensorboard_dir='log')

In [ ]:
def label_img(img_name):
  if 'cat' in img_name: return [1, 0]
  elif 'dog' in img_name: return [0, 1]

In [ ]:
training_data = []

with zipfile.ZipFile('/content/drive/MyDrive/train.zip', 'r') as z:
  for fn in tqdm(z.namelist()):
    if not os.path.isdir(fn) and fn.endswith("jpg"):
      label = label_img(fn)

      img_str = z.read(fn)

      nparr = np.frombuffer(img_str, np.uint8)

      img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
      img = cv2.resize(img_np, (IMG_WIDTH, IMG_HEIGHT))

      training_data.append([ np.array(img), np.array(label)])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/25001 [00:00<?, ?it/s]

In [ ]:
train_size = int(len(training_data) * .8)
train_size

train = training_data[-train_size:]
display('Train Size: {}'.format(len(train)))

test = training_data[:-train_size]
display('Test Size: {}'.format(len(test)))

X_train = np.array([i[0] for i in train]).reshape(-1,IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS)
y_train = np.array([i[1] for i in train])

X_test = np.array([i[0] for i in test]).reshape(-1,IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS)
y_test = np.array([i[1] for i in test])

'Train Size: 20000'

'Test Size: 5000'

In [ ]:
reset_graph()

model_baseline = get_model_baseline()

start_time = time.clock()

model_baseline.fit({'input': X_train}, {'targets': y_train}, n_epoch=3, validation_set=({'input': X_test}, {'targets': y_test}),
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

end_time = time.clock()

runtime = end_time - start_time  # seconds of wall-clock time

Training Step: 938  | total loss: 8.61262 | time: 11.198s
| Adam | epoch: 003 | loss: 8.61262 - acc: 0.6260 -- iter: 19968/20000
Training Step: 939  | total loss: 8.39896 | time: 12.439s
| Adam | epoch: 003 | loss: 8.39896 - acc: 0.6352 | val_loss: 23.02585 - val_acc: 0.0000 -- iter: 20000/20000
--


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
reset_graph()

model_v2 = get_model_v2()

start_time = time.clock()

model_v2.fit({'input': X_train}, {'targets': y_train}, n_epoch=3, validation_set=({'input': X_test}, {'targets': y_test}),
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

end_time = time.clock()

runtime = end_time - start_time  # seconds of wall-clock time

Training Step: 751  | total loss: 0.46702 | time: 4.615s
| Adam | epoch: 003 | loss: 0.46702 - acc: 0.7723 -- iter: 08000/20000


In [ ]:
training_data2 = []

with zipfile.ZipFile('/content/drive/MyDrive/train.zip', 'r') as z:
  for fn in tqdm(z.namelist()):
    if not os.path.isdir(fn) and fn.endswith("jpg"):
      label = label_img(fn)

      img_str = z.read(fn)

      nparr = np.frombuffer(img_str, np.uint8)

      img_np = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
      img = cv2.resize(img_np, (IMG_WIDTH, IMG_HEIGHT))

      training_data2.append([ np.array(img), np.array(label)])

In [ ]:
train_size2 = int(len(training_data2) * .8)
train_size2

train2 = training_data2[-train_size2:]
display('Train Size: {}'.format(len(train2)))

test2 = training_data2[:-train_size2]
display('Test Size: {}'.format(len(test2)))

X_train2 = np.array([i[0] for i in train2]).reshape(-1,IMG_WIDTH, IMG_HEIGHT, 1)
y_train2 = np.array([i[1] for i in train2])

X_test2 = np.array([i[0] for i in test2]).reshape(-1,IMG_WIDTH, IMG_HEIGHT, 1)
y_test2 = np.array([i[1] for i in test2])

In [ ]:
training_data2[0][0].shape

In [ ]:
def get_model_baseline2():
  convnet = input_data(shape=[None, IMG_WIDTH, IMG_HEIGHT, 1], name='input')

  convnet = conv_2d(convnet, 32, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = conv_2d(convnet, 64, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = fully_connected(convnet, 1024, activation='relu')
  convnet = dropout(convnet, 0.8)

  convnet = fully_connected(convnet, 2, activation='softmax')
  convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

  return tflearn.DNN(convnet, tensorboard_dir='log')



In [ ]:
def get_model_v2_2():
  convnet = input_data(shape=[None, IMG_WIDTH, IMG_HEIGHT, 1], name='input')

  convnet = conv_2d(convnet, 32, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = conv_2d(convnet, 64, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = conv_2d(convnet, 128, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = conv_2d(convnet, 64, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = conv_2d(convnet, 32, 5, activation='relu')
  convnet = max_pool_2d(convnet, 5)

  convnet = fully_connected(convnet, 1024, activation='relu')
  convnet = dropout(convnet, 0.8)

  convnet = fully_connected(convnet, 2, activation='softmax')
  convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

  return tflearn.DNN(convnet, tensorboard_dir='log')

In [ ]:
reset_graph()

model_baseline2 = get_model_baseline2()

start_time = time.clock()

model_baseline2.fit({'input': X_train2}, {'targets': y_train2}, n_epoch=3, validation_set=({'input': X_test2}, {'targets': y_test2}),
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

end_time = time.clock()

runtime = end_time - start_time  # seconds of wall-clock time

In [ ]:
reset_graph()

model_v2_2 = get_model_v2_2()

start_time = time.clock()

model_v2_2.fit({'input': X_train2}, {'targets': y_train2}, n_epoch=3, validation_set=({'input': X_test2}, {'targets': y_test2}),
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

end_time = time.clock()

runtime = end_time - start_time  # seconds of wall-clock time

In [ ]:
test_data = []

with zipfile.ZipFile('/content/drive/MyDrive/test.zip', 'r') as z:
  for fn in tqdm(z.namelist()):
    if not os.path.isdir(fn) and fn.endswith("jpg"):
      label = label_img(fn)

      img_str = z.read(fn)

      nparr = np.frombuffer(img_str, np.uint8)

      img_np = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
      img = cv2.resize(img_np, (IMG_WIDTH, IMG_HEIGHT))

      test_data.append([ np.array(img), np.array(label)])

In [ ]:
test_data2 = []

with zipfile.ZipFile('/content/drive/MyDrive/test.zip', 'r') as z:
  for fn in tqdm(z.namelist()):
    if not os.path.isdir(fn) and fn.endswith("jpg"):
      label = label_img(fn)

      img_str = z.read(fn)

      nparr = np.frombuffer(img_str, np.uint8)

      img_np = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
      img = cv2.resize(img_np, (IMG_WIDTH, IMG_HEIGHT))

      test_data2.append(np.array(img))

In [ ]:
model_baseline2.predict(test_data2.reshape(-1, 1))

In [ ]:
for data, label in test_data2:
  print(data)
  break

In [ ]:
!unzip /content/drive/MyDrive/train.zip

In [ ]:
!unzip /content/drive/MyDrive/test.zip

In [ ]:
TRAIN_DIR = 'train/'
TEST_DIR = 'test/'
train_dogs = ['./train/{}'.format(i) for i in os.listdir(TRAIN_DIR) if 'dog' in i]
train_cats = ['./train/{}'.format(i) for i in os.listdir(TRAIN_DIR) if 'cat' in i]
test_images_dogs_cats = ['./test/{}'.format(i) for i in os.listdir(TEST_DIR)]
len(train_dogs), len(train_cats), len(test_images_dogs_cats)

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

train_dogs.sort(key=natural_keys)
train_cats.sort(key=natural_keys)
train_images_dogs_cats = train_dogs[0:1300] + train_cats[0:1300]

test_images_dogs_cats.sort(key=natural_keys)
len(train_images_dogs_cats), len(test_images_dogs_cats)


In [ ]:
img_width = 150
img_height = 150
def prepare_data(list_of_images):
    """
    Returns two arrays:
        x is an array of resized images
        y is an array of labels
    """
    x = [] # images as arrays
    y = [] # labels

    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC))

    for i in list_of_images:
        if 'dog' in i:
            y.append(1)
        elif 'cat' in i:
            y.append(0)
    return x, y

X, Y = prepare_data(train_images_dogs_cats)
XT, YT = prepare_data(test_images_dogs_cats)
print(K.image_data_format())
# Since K.image_data_format() is channel_last,
# input_shape to the first keras layer will be (img_width, img_height, 3).
# '3' since it is a color image

In [ ]:
CD_train= 'train'
CD_test= 'test'

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X,Y, test_size=0.2, random_state=1)
X_test = XT
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 32

In [ ]:
X_train[0].shape, X_val[0].shape, X_test[0].shape

In [ ]:
def gen_image_label(directory):
    ''' A generator that yields (label, id, jpg_filename) tuple.'''
    for root, dirs, files in os.walk(directory):
        for f in files:
            _, ext = os.path.splitext(f)
            if ext != '.jpg':
                continue
            basename = os.path.basename(f)
            splits = basename.split('.')
            if len(splits) == 3:
                label, id_, ext = splits
            else:
                label = None
                id_, ext = splits
            fullname = os.path.join(root, f)
            yield label, int(id_), fullname

In [ ]:
# Wrap testing data into pandas' DataFrame.
lst = list(gen_image_label(CD_test))
CD_test_df = pd.DataFrame(lst, columns=['label', 'id', 'filename'])
CD_test_df = CD_test_df.sort_values(by=['label', 'id'])
CD_test_df['label_code'] = CD_test_df.label.map({'cat':0, 'dog':1})

CD_test_df.head(3)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow(np.array(X_train), Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(np.array(X_val), Y_val, batch_size=batch_size)

In [ ]:
model1A = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape=[150, 150, 3]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation="softmax")
])

model1A.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])



In [ ]:
%time
history = model1A.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

In [ ]:
y_pred_1A= model1A.predict(X_test)

In [ ]:
#removed dropout

model2A = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape=[150, 150, 3]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    #keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    #keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    #keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model2A.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])



In [ ]:
%time
history = model2A.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

In [ ]:
y_pred_2A= model2A.predict(X_test)

In [ ]:
#B series models have more nodes and layers

model1B = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape=[150, 150, 3]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation="softmax")
])
model1B.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])



In [ ]:
%time
history = model1B.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

In [ ]:
%time history = model1B.fit(np.array(X_train), np.array(Y_train), epochs=10,\
                            validation_data=(np.array(X_val), np.array(Y_val)))

In [ ]:
y_pred_1B= model1B.predict(X_test)

In [ ]:
#removed dropout
#Cut filters for Conv Layers in half

model2B = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                        input_shape=[150, 150, 3]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

model2B.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])



In [ ]:
%time history = model2B.fit(np.array(X_train), np.array(Y_train), epochs=10,\
                            validation_data=(np.array(X_val), np.array(Y_val)))

In [ ]:
y_pred_2B= model2B.predict(X_test)